In [ ]:
spx = get_pricing(["spy","aapl"],  start_date='2014-06-01', end_date='2014-10-01')
spx.transpose(2,1,0)[symbols("spy")].open_price.plot()
# spx.plot()
# spx.rolling(window=20).mean().plot()


![](https://www.evernote.com/shard/s6/sh/b71dce8f-1009-48d4-8d0f-a55f92c3b4c2/e7aaab58314c9f2c13a539b048cb7086/res/6b142e38-5546-4d87-827b-7794f99a50ea/%E3%83%80%E3%82%A6%E3%83%B3%E3%83%AD%E3%83%BC%E3%83%89%20(5).png)

作戦。

移動平均より下にあれば上がる、上にあれば下がるという考え。
`Factor=現在価格/移動平均` とおくと、より大きいFactorを**売り**、より小さいFactorを**買う**ということになる。

しかし、Quantopianの `opt.MaximizeAlpha()` をオーダーの時に使う必要があり、
ファクターが大きいものを買い、ファクターが小さいものを売るというオーダーになるので、
今回のFactorの計算は、`Factor = 移動平均/現在価格` 逆数を渡してFactorとする。



In [ ]:
# https://note.mu/notes/nc229144d5f76/ 用
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline import Pipeline

from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.pipeline.classifiers.fundamentals import Sector  


import alphalens as al 


MORNINGSTAR_SECTOR_CODES = {
     -1: 'Misc',
    101: 'Basic Materials',
    102: 'Consumer Cyclical',
    103: 'Financial Services',
    104: 'Real Estate',
    205: 'Consumer Defensive',
    206: 'Healthcare',
    207: 'Utilities',
    308: 'Communication Services',
    309: 'Energy',
    310: 'Industrials',
    311: 'Technology' ,    
}


def my_sma():
    
    # contest 用 ユニバース
    base_universe= QTradableStocksUS()
    
    sma = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=20)
    adj_close = USEquityPricing.close.latest 
    sector = Sector()

    # 上記書いたとおり
    myfactor = sma / adj_close 
    
    # 各日、各銘柄毎のmyfactorをzscoreでランキングする
    # その時使う銘柄はbase_universeに入っている銘柄だけ
    zscored_factor = myfactor.zscore(mask=base_universe)
    
    return Pipeline(
        columns = {
            'zscored_factor':zscored_factor, 
            'adj_close':adj_close,
            'sma':sma,
            'Sector' : Sector(),
            'myfactor':myfactor,
        },
        screen=zscored_factor.notnull() 
    )

results = run_pipeline(my_sma(), '2016-01-01', '2018-01-01')



In [ ]:
## 結果を見てみましょう
results.head()


In [ ]:
## run_pipelineした期間と同じ期間の株式データを取得する。
asset_list = results.index.levels[1]
prices = get_pricing(asset_list, start_date='2016-01-01', end_date='2018-01-01', fields='close_price')
prices.head()


In [ ]:
## alphalens でFactorと収益の関係を探る
## alphalens では将来の収益に対して、今日のFactorがどのくらいPredictable（予測可能な）ファクターであるかを調べる事ができます。
## そのために、何日後の収益と比べたいかをalphalensに与える必要があるので、それを指定します。下記の様に複数日可です。
periods = (1,5,10)

## alphalens にファクターを渡して分析する。
## 各ファクターをランキングするために作ったzscored_factorをファクターとして渡し、それがどれくらい予測可能なのかも分析する
factor_data = al.utils.get_clean_factor_and_forward_returns(
                factor=results["zscored_factor"],
                prices=prices,
                groupby=results["Sector"],
                groupby_labels=MORNINGSTAR_SECTOR_CODES,
                periods=periods,
                quantiles=10)

In [ ]:
factor_data.head()

In [ ]:
al.tears.create_returns_tear_sheet(factor_data, by_group=True)
## まずはReturns Analysisというまとめが出てきます。
## 指定した日数に対して、アルファ、ベータ、そして、私達のFactorのTopQuantileとBottomQuantileのBasis Pointでのリターン
## それのスプレッドを出してくれます。
## 結果も感じです(^o^)

ではアルゴリズムを書きましょう。
フォーラムやTutorialを探すとベースになるようなアルゴリズムがたくさん有りますので
色々悩まず、先人のお力を借ります。
https://www.quantopian.com/lectures/example-long-short-equity-algorithm




```python
"""
finpy 資料
"""
from quantopian.pipeline.data.builtin import USEquityPricing

import quantopian.algorithm as algo
import quantopian.optimize as opt
from quantopian.pipeline import Pipeline
from quantopian.pipeline.factors import SimpleMovingAverage

from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.experimental import risk_loading_pipeline


# レバレッジはコンテストの規則で1.1までだがバッファで1.0に。
MAX_GROSS_LEVERAGE = 1.0
# 全ポジションは600まで。投入資金が少なければMaxに届かない事もある。
TOTAL_POSITIONS = 600

# Here we define the maximum position size that can be held for any
# given stock. If you have a different idea of what these maximum
# sizes should be, feel free to change them. Keep in mind that the
# optimizer needs some leeway in order to operate. Namely, if your
# maximum is too small, the optimizer may be overly-constrained.

# 保有銘柄数をコントロールするための定数。
# 最適なオーダーのために多少の余裕をもたせたほうがいいと書いてあるので、
# TOTAL_POSITIONS = 600のままにした。
MAX_SHORT_POSITION_SIZE = 2.0 / TOTAL_POSITIONS
MAX_LONG_POSITION_SIZE = 2.0 / TOTAL_POSITIONS


def initialize(context):
   
    algo.attach_pipeline(make_pipeline(), 'long_short_equity_template')
    algo.attach_pipeline(risk_loading_pipeline(), 'risk_factors')

    algo.schedule_function(func=rebalance,
                           date_rule=algo.date_rules.week_start(),
                           time_rule=algo.time_rules.market_close(),
                           half_days=False)

    algo.schedule_function(func=record_vars,
                           date_rule=algo.date_rules.every_day(),
                           time_rule=algo.time_rules.market_close(),
                           half_days=True)


def make_pipeline():
    sma = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=20)
    adj_close = USEquityPricing.close.latest 
    myfactor = sma / adj_close
    quantiled_myfactor = myfactor.quantiles(10)
    
    universe = QTradableStocksUS()
    # 外れ値をはずしてみた。
    myfactor_windorized = myfactor.winsorize(min_percentile=0.10, max_percentile=0.90)

    # 今後、新しいFactorを追加してつかう事を想定してcombined_factorを作っている
    combined_factor = (
        myfactor_windorized.zscore() 
        # + another_myfactor.zscore() 例えば、こういうふうに、+でつなげて追加すればよい
    )

    # combined_factorの上からと下からTOTAL_POSITIONS//2銘柄取得。それをロングとショートする。
    longs= combined_factor.top(TOTAL_POSITIONS//2, mask=universe )
    shorts= combined_factor.bottom(TOTAL_POSITIONS//2, mask=universe )
    long_short_screen = (longs | shorts)

    # Create pipeline
    pipe = Pipeline(
        columns={
            'longs': longs,
            'shorts': shorts,
            'combined_factor': combined_factor,
            'myfactor':myfactor, 
            'quantiled_myfactor':quantiled_myfactor,
            
        },
        screen=long_short_screen
    )
    return pipe


def before_trading_start(context, data):
    # pipeline 作成
    context.pipeline_data = algo.pipeline_output('long_short_equity_template')
   
    # リスクファクターを計算するパイプラインを作成
    context.risk_loadings = algo.pipeline_output('risk_factors')

def record_vars(context, data):
    # ポジション数を描画する
    algo.record(num_positions=len(context.portfolio.positions))


def rebalance(context, data):
    """
    毎日のリバランスをする関数。
    """
    # パイプラインからデータを取得。 pandas DataFrame 型
    pipeline_data = context.pipeline_data
    risk_loadings = context.risk_loadings

    # ファクターを最大限に使いつつ最適化（factorが大きい銘柄群をLong、小さい銘柄群をShortする）して、
    # 銘柄を選ぶ関数を使いオブジェクトを取得。
    # Optimize API については、https://www.quantopian.com/help#optimize-title を参照してください。
    objective = opt.MaximizeAlpha(pipeline_data.combined_factor)

    # objective に渡す「制限リスト」を作成。制限を渡さないと、当てはまるすべての銘柄を取引してしまう。
    # constraint については、 https://www.quantopian.com/help#module-quantopian_optimize_constraints 
    constraints = []
    # レバレッジが1.0になるように制限
    constraints.append(opt.MaxGrossExposure(MAX_GROSS_LEVERAGE))

    # ダラーニュートラル(long/short両サイドが同じサイズ)になるように制限
    constraints.append(opt.DollarNeutral())

    # Add the RiskModelExposure constraint to make use of the
    # default risk model constraints
    # すみません。ここは勉強不足でわかりません
    neutralize_risk_factors = opt.experimental.RiskModelExposure(
        risk_model_loadings=risk_loadings,
        version=0
    )
    constraints.append(neutralize_risk_factors)

    # 設定したポジションサイズを超えない様にする。
    # かつ、一つもしくは少数の銘柄だけのポジションをつくらないようにする。
    constraints.append(
        opt.PositionConcentration.with_equal_bounds(
            min=-MAX_SHORT_POSITION_SIZE,
            max=MAX_LONG_POSITION_SIZE
        ))

    # objective と 制約リストを渡して注文する
    algo.order_optimal_portfolio(
        objective=objective,
        constraints=constraints
    )
    ```
    
![](https://www.evernote.com/shard/s6/res/cafddff0-51d7-41ee-bccc-50b0a88c5249/2018-11-28_005.jpg?)    

コケたところを見てみましょう

![](https://www.evernote.com/shard/s6/res/c224377d-04af-4b40-b752-4e332941f38e/Screenshot%20from%202018-11-28%2018-52-44.png)
Short Term Reversalに引っかかってるよ、て言われましたが、なんすか？と思って調べたら大変なことに気が付きました

[Quantopian Risk Model](https://www.quantopian.com/papers/risk)ここで、

Style Factor Definitions→Short-term reversalの説明によると

`Short-term reversal: The short-term reversal factor captures the difference in returns between stocks with strong recent losses theoretically primed to reverse (recent loser stocks) and stocks with strong recent gains theoretically primed to reverse (recent winner stocks) in a short time period.`

ほわっ？？？？？



どうも、直近ですごく負けてた銘柄がグンっと上がって、すごく勝ってた銘柄がグンっと負けたことで得る収益もQuantopianのリスクファクターにカウントされていて，それが±40％以上（以下）だとNGということのようですね．

![](https://emojis.slackmojis.com/emojis/images/1457563042/312/doge.png)

そういうストラテジー作った．．．よ．．．ね．．．？

や，やり直しか？！

時間がない（FinTech忘年会まで）．となりましたが，試行錯誤の結果 `sma の window=60` に変えたら通りました＼(^o^)／ アドホック＼(^o^)／

![](https://www.evernote.com/shard/s6/res/8f6882bc-8223-43e5-9d02-a4e792ab395d/2018-11-29_14.jpg?resizeSmall&width=1308)

では `Enter Contest`でコンテストにJoin. Entry Nameを入れたらSubmitします．

![](https://www.evernote.com/shard/s6/res/cd795f5f-f00b-4c75-b3c2-c9bb364c1f44/2018-11-29_18.jpg)

submit するとこんなメッセージが出ます．

> You currently have 1 active/pending contest entry. You may submit up to 3 more entries. Submit another algorithm for more chances to win.
Your pending entries will be evaluated and scored after the next trading day. You will receive an email when your entry has been evaluated. Check your active entries to see your score after the evaluation.

> コンテストに参加しているエントリーが一つあります．（アクティブかペンディングかの状態で）．3つまで応募できるんでやってみて．
ペンディングされてるエントリーは，応募した日の次の営業日には評価されます．無事エントリーできたかは，メールでお知らせするんで待ってて．

（サブミットと同時に同じ内容のメールも来ます．それはアクセプトメールじゃないのでご注意を．）

緊張しますね．ちゃんとアクセプトされてほしい！



